In [ ]:
Daniela Fabrega
Felipe Saldias

# Algoritmo Metropolis Hastings

## Problema de la mochila


Dado un conjunto de $m$ elementos cada uno descrito por su masa $w_j$ y su valor $v_j$ y una mochila cuyo límite de capacidad es $C$. Asumiendo que el volumen y la forma de los objetos no importan, encuentre el subconjunto de objetos de mayor valor que puede ser llevado en la mochila. Este es un problema de optimización combinatorial NP completo

Podemos definir la variable indicadora

$$
x = (z_1, z_2, \ldots, z_m)
$$

donde cada $z_i \in \{0, 1\}$ es igual a 1 si el elemento $i$ está en la mochila y 0 si no lo está

Se define entonces un espacio de posibilidades

$$
\Omega = \left \{x \in {0, 1}^m : \sum_{j=1}^m w_j z_j \leq C \right \}
$$

De donde queremos encontrar aquella que maximiza la utilidad

$$
U(x) = \sum_{j=1}^m v_j z_j
$$

## Solución con Monte Carlo

Para resolverlo con Monte Carlo podríamos

- Dado $x_t$
- Escoger $j \in [1, ..., m]$ al azar de manera uniforme
- Crear $y=(z_1, z_2, ..., 1-z_j,..., z_m)$, si es la mejor solución hasta ahora, guardarla
- Si $y$ es factible entonces $x_{t+1} = y$ de lo contrario $x_{t+1} = x_t$

Pero esto podría tardar muchisimo para $m$ grande

## Actividad: Simulated Annealing

Resuelva este problema usando el algoritmo de Simulated Annealing, el cual es una versión del algoritmo de Metropolis donde la distribución de interés se formula a partir de la función de utilidad como

$$
p(x) = \frac{1}{Z} \exp \left(\frac{U(x)}{T} \right) 
$$

donde $T$ es la temperatura y $Z = \sum_{x\in \Omega}  \exp \left(\frac{U(x)}{T} \right)$ es la evidencia que no depende de $x$

- Escriba la taza de aceptación $\alpha$ y el valor de $r$
- Implemente la solución de Monte Carlo 
- Implemente la solución de Simulated annealing modificando el criterio de aceptación ingenuo por de Metropolis
- Para el conjunto de datos $X_1$
    - Compare SA con el método de montecarlo clásico ¿Cuánto demora cada uno en llegar a la solución óptima?
    - Explore la influencia del parámetro $T$. Muestre y compare resultados con un $T$ grande, adecuado y pequeño decididos por usted. Pruebe con un valor de $T$ adaptivo dado por
    $$
    T_i = \frac{1}{\log(i)}
    $$
- Para el conjunto de datos $X_2$. Encuentre un valor de $T$ adecuado y muestre la mejor solución obtenida usando SA

Referencias:

- Láminas 17 a 21: https://cindy.informatik.uni-bremen.de/cosy/teaching/CM_2011/fitting/mcmc.pdf
- Láminas 4 a 8: http://sites.science.oregonstate.edu/~kovchegy/web/papers/MCMC.pdf



In [2]:
X1 = {"m": 10, "C": 2586,
      "v": [81, 141, 211, 321, 408, 549, 596, 750, 953, 1173], 
      "w": [36, 78, 110, 214, 259, 356, 377, 487, 689, 862]
     }

X2 = {"m": 25, "C": 10356,
      "v": [39, 93, 159, 240, 274, 493, 588, 752, 1025, 1324, 1588, 1826, 1936, 2045, 
            2287, 2486, 2818, 2850, 3072, 3219, 3499, 3596, 3620, 4067, 4432], 
      "w": [5, 42, 84, 126, 133, 309, 368, 502, 761, 1020, 1283, 1517, 1584, 1656, 
            1865, 2031, 2320, 2349, 2553, 2667, 2929, 3024, 3047, 3452, 3790]
     }

In [51]:
def calculate_weight_value(x_set):
    x_set_weight_sum = 0
    x_set_value_sum = 0
    for x_prop in x_set:
        x_set_weight_sum += X1["w"][x_prop]
        x_set_value_sum += X1["v"][x_prop]    
    return x_set_weight_sum, x_set_value_sum
    

In [300]:
import random 


weight_limit = X1["C"]
def montecarlo(iterations):
    
    best_for_epoch=[]
    the_best_one = {"sum": 0,
               "array":[0,0,0]}
    
    x_i = []

    for i in range(iterations):
        element = random.randint(0,9)
        x_propuesta=x_i.copy()

        if element not in x_propuesta:
            x_propuesta.append(element)
        else:
            x_propuesta.remove(element)   

        x_propuesta_weight_sum, x_propuesta_value_sum = calculate_weight_value(x_propuesta)

        if x_propuesta_weight_sum <= weight_limit:
            x_i = x_propuesta

            if(x_propuesta_value_sum > the_best_one["sum"]):
                the_best_one["array"] = x_propuesta.copy()
                the_best_one["sum"] = x_propuesta_value_sum
        best_for_epoch.append(the_best_one["sum"])
    print("El mejor resultado", the_best_one)
    return(the_best_one, best_for_epoch)

## annealing

In [301]:
import random 
import math
import numpy as np

def annealing(iterations, T=1):

    best_for_epoch=[]

    the_best_one = {"sum": 0,
                   "array":[0,0,0]}

    weight_limit = X1["C"]
    x_i = []

    for i in range(iterations):
        element = random.randint(0,9)
        x_propuesta=x_i.copy()

        if element not in x_propuesta:
            x_propuesta.append(element)
        else:
            x_propuesta.remove(element)   

        x_propuesta_weight_sum, x_propuesta_value_sum = calculate_weight_value(x_propuesta)
        x_current_weight_sum, x_current_value_sum = calculate_weight_value(x_i)

        if x_propuesta_weight_sum <= weight_limit:
            r= np.exp(x_propuesta_value_sum/T)/np.exp(x_current_value_sum/T)
            alpha = min(1,r)
            if alpha > random.uniform(0, 1):
                x_i = x_propuesta

            if(x_propuesta_value_sum > the_best_one["sum"]):
                the_best_one["array"] = x_propuesta.copy()
                the_best_one["sum"] = x_propuesta_value_sum
                
        best_for_epoch.append([i,the_best_one["sum"]])
    print("El mejor resultado", the_best_one)
    return(the_best_one, best_for_epoch)

In [302]:
montecarlo(6)

El mejor resultado {'sum': 1583, 'array': [0, 8, 1, 4]}


({'sum': 1583, 'array': [0, 8, 1, 4]}, [81, 1034, 1175, 1583, 1583, 1583])

In [296]:
annealing(6)


El mejor resultado {'sum': 1878, 'array': [2, 3, 7, 6]}


<ipython-input-214-716d0ddca15f>:28: RuntimeWarning: overflow encountered in exp
  r= np.exp(x_propuesta_value_sum/T)/np.exp(x_current_value_sum/T)
<ipython-input-214-716d0ddca15f>:28: RuntimeWarning: invalid value encountered in double_scalars
  r= np.exp(x_propuesta_value_sum/T)/np.exp(x_current_value_sum/T)


{'sum': 1878, 'array': [2, 3, 7, 6]}

In [123]:
r= math.exp(750/T)/math.exp(0/T)

OverflowError: math range error

In [130]:
import numpy

In [132]:
numpy.exp(750/2)

7.251547794405553e+162

In [127]:
T=100

In [128]:
math.exp(750/T)

1808.0424144560632

In [124]:
math.exp(0/T)

1.0

In [119]:
x,y=calculate_weight_value([])

0.01831563888873418

In [4]:
partial_index_array

[6, 0, 5, 8]

In [13]:
L = X1["w"]

In [15]:
L[partial_index_array]

TypeError: list indices must be integers or slices, not list